In [ ]:
!pip install -U finance-datareader
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import FinanceDataReader as fdr

In [2]:
통합재무정보 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/statements.csv', index_col = 0,  dtype={'종목코드':'str'})

# 데이터 가져옴

In [43]:
자산총계누락 = 통합재무정보[(통합재무정보['자산총계'] == '0')|(통합재무정보['자산총계'] == 0)]['종목코드'].unique().tolist()
부채총계누락 = 통합재무정보[(통합재무정보['부채총계'] == '0')|(통합재무정보['부채총계'] == 0)]['종목코드'].unique().tolist()
자본총계누락 = 통합재무정보[(통합재무정보['자본총계'] == '0')|(통합재무정보['자본총계'] == 0)]['종목코드'].unique().tolist()

매출액누락 = 통합재무정보[(통합재무정보['매출액'] == '0')|(통합재무정보['매출액'] == 0)]['종목코드'].unique().tolist()
영업이익누락 = 통합재무정보[(통합재무정보['영업이익'] == '0')|(통합재무정보['영업이익'] == 0)]['종목코드'].unique().tolist()
매출총이익누락 = 통합재무정보[(통합재무정보['매출총이익'] == '0')|(통합재무정보['매출총이익'] == 0)]['종목코드'].unique().tolist()
당기순이익누락 = 통합재무정보[(통합재무정보['당기순이익'] == '0')|(통합재무정보['당기순이익'] == 0)]['종목코드'].unique().tolist()
법인세차감전순이익누락 = 통합재무정보[(통합재무정보['법인세차감전순이익'] == '0')|(통합재무정보['법인세차감전순이익'] == 0)]['종목코드'].unique().tolist()


전체누락 = 자산총계누락 + 부채총계누락 + 자본총계누락 + 매출액누락 + 영업이익누락 + 매출총이익누락 + 당기순이익누락 + 법인세차감전순이익누락

KeyError: ignored

In [4]:
print(f"자산총계누락: {len(자산총계누락)}")
print(f"부채총계누락: {len(부채총계누락)}")
print(f"자본총계누락: {len(자본총계누락)}")
print(f"매출액누락: {len(매출액누락)}")
print(f"영업이익누락: {len(영업이익누락)}")
print(f"매출총이익누락: {len(매출총이익누락)}")
print(f"당기순이익누락: {len(당기순이익누락)}")

print()
print('=========================')
print(f"전체누락: {len(전체누락)}")

자산총계누락: 15
부채총계누락: 3
자본총계누락: 5
매출액누락: 133
영업이익누락: 58
매출총이익누락: 184
당기순이익누락: 172

전체누락: 570


In [5]:
전체누락_unq = list(set(전체누락))
print(f"전체누락 unique: {len(전체누락_unq)}")

전체누락 unique: 381


In [6]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [7]:
포괄손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(포괄손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  포괄손익계산샘플 = 포괄손익계산샘플.append(temp)

포괄손익계산샘플['종목코드'] = 포괄손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  손익계산샘플 = 손익계산샘플.append(temp)

손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

  4%|▍         | 1/24 [00:00<00:17,  1.34it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 24/24 [00:01<00:00, 16.68it/s]


In [13]:
매출액누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(매출액누락)*통합재무정보['매출액'] == '0')][['종목코드','사업연도','보고서종류','매출액']]
매출액누락_정보 = 매출액누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
영업이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(영업이익누락))&(통합재무정보['영업이익'] == '0')][['종목코드','사업연도','보고서종류','영업이익']]
영업이익누락_정보 = 영업이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
매출총이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(매출총이익누락))&(통합재무정보['매출총이익'] == '0')][['종목코드','사업연도','보고서종류','매출총이익']]
매출총이익누락_정보 = 매출총이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
당기순이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(당기순이익누락))&(통합재무정보['당기순이익'] == '0')][['종목코드','사업연도','보고서종류', '당기순이익']]
당기순이익누락_정보 = 당기순이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])

In [55]:
df = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(당기순이익누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df = df.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df = df.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df

747it [04:01,  3.09it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
1238,950170,2018,1분기보고서,JTC,471,ifrs_ComprehensiveIncome,총포괄손익,NaN
1239,950170,2018,1분기보고서,JTC,471,entity01041828_udf_IS_20187231720157_Comprehen...,지배기업의 소유주지분순이익,"486,129,255"
1240,950170,2018,1분기보고서,JTC,471,ifrs_EarningsPerShareAbstract,주당이익 [abstract],NaN
1241,950170,2018,1분기보고서,JTC,471,ifrs_BasicEarningsLossPerShare,기본주당이익(손실) (엔),14


In [61]:
df = df.replace(0, np.NaN)
df = df.replace('0', np.NaN)
df.dropna(axis=0, inplace=True)
df

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
1236,950170,2018,1분기보고서,JTC,471,dart_OtherComprehensiveIncomeThatWillBeReclass...,당기손익으로 재분류될 수 있는 항목(세후기타포괄손익),"24,883,013"
1237,950170,2018,1분기보고서,JTC,471,ifrs_GainsLossesOnExchangeDifferencesOnTransla...,해외사업장환산외환차이(세후기타포괄손익),"24,883,013"
1239,950170,2018,1분기보고서,JTC,471,entity01041828_udf_IS_20187231720157_Comprehen...,지배기업의 소유주지분순이익,"486,129,255"
1241,950170,2018,1분기보고서,JTC,471,ifrs_BasicEarningsLossPerShare,기본주당이익(손실) (엔),14


In [68]:
df[(df['항목명'].str.contains('순이익'))|(df['항목코드'].str.contains('ProfitLoss'))]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31452,000440,2017,반기보고서,중앙에너비스,467,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익(손실),"1,260,250,690"
31454,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781119916571_Statemen...,계속영업당기순이익(손실),"960,203,538"
31455,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201788113432879_Statemen...,중단영업당기순이익(손실),"149,724,109"
2051,000680,2017,반기보고서,LS네트웍스,468,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익(손실),"17,952,354,341"
2053,000680,2017,반기보고서,LS네트웍스,468,ifrs_ProfitLossFromContinuingOperations,계속영업이익(손실),"10,867,958,930"
...,...,...,...,...,...,...,...,...
32382,900310,2017,반기보고서,컬러레이,715,entity01236286_udf_IS_201782410187124_Statemen...,반기순이익,"69,885,510"
32385,900310,2017,반기보고서,컬러레이,715,ifrs_ProfitLossAttributableToOwnersOfParent,지배기업의 소유주에게 귀속되는 당기순이익,"69,885,510"
1231,950170,2018,1분기보고서,JTC,471,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익(손실),"756,684,636"
1234,950170,2018,1분기보고서,JTC,471,entity01041828_udf_IS_2018723171925815_ProfitL...,지배기업의 소유주지분순이익,"461,246,242"


In [77]:
df[(df['항목명'].isin(['당기순이익', '당기순이익(손실)', '순이익', '순이익(손실)','당기순이익(순손실)', '순이익(순손실)']))]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
17318,001430,2015,사업보고서,세아베스틸,241,entity00106669_udf_IS_20155292156891_Statement...,당기순이익(손실),"165,047,731,489"
17378,001430,2016,1분기보고서,세아베스틸,241,entity00106669_udf_IS_20155292156891_Statement...,당기순이익(손실),"28,834,278,944"
17721,001430,2016,3분기보고서,세아베스틸,241,entity00106669_udf_IS_20155292156891_Statement...,당기순이익(손실),"84,302,308,344"
17604,001430,2016,반기보고서,세아베스틸,241,entity00106669_udf_IS_20155292156891_Statement...,당기순이익(손실),"64,738,318,662"
18107,001430,2016,사업보고서,세아베스틸,241,entity00106669_udf_IS_20155292156891_Statement...,당기순이익(손실),"104,654,832,540"
...,...,...,...,...,...,...,...,...
40598,213500,2016,사업보고서,한솔제지,171,entity01060744_udf_IS_20135139349843_Statement...,당기순이익,"42,599,085,289"
40033,213500,2017,1분기보고서,한솔제지,171,entity01060744_udf_IS_20135139349843_Statement...,당기순이익,"19,375,674,896"
39535,213500,2017,반기보고서,한솔제지,171,entity01060744_udf_IS_20135139349843_Statement...,당기순이익,"52,246,171,044"
21194,214430,2017,1분기보고서,아이쓰리시스템,262,entity00612489_udf_IS_2017511155640471_Stateme...,당기순이익,"1,219,881,215"


In [73]:
(df[(df['항목명'].str.contains('당기순이익'))|(df['항목코드'].str.contains('ProfitLoss'))]['항목명'].unique())

array(['법인세비용차감전순이익(손실)', '계속영업당기순이익(손실)', '중단영업당기순이익(손실)', '계속영업이익(손실)',
       '지배기업의 소유주에게 귀속되는 당기순이익(손실)', '비지배지분에 귀속되는 당기순이익(손실)',
       '법인세비용차감전이익(손실)', '지배기업 소유주지분', '비지배지분', '지분법이익', '당기순이익(손실)',
       '지분법손익', '관계기업 및 공동기업에 대한 지분법손익', '법인세비용차감전순이익', '지배주주지분',
       '중단영업이익(손실)', '연결당기순이익', '지분법 적용대상인 관계기업과 조인트벤처의 당기순손익에 대한 지분',
       '당기순이익', '지배기업소유주지분', '연결당기순이익(순손실)', '지배기업주주지분', '당기순이익(순손실)',
       '지배기업의 소유주지분', '지배기업 귀속 반기순이익', '비지배지분 귀속 반기순이익', '지배기업의 소유주',
       '계속영업당기순손익', '중단영업단기순손익', '법인세비용차감전이익', '계속영업분기순이익',
       '후속적으로 당기순이익으로 재분류 되지않는 항목', '후속적으로 당기순이익으로 재분류 되는 항목',
       '지배기업소유주지분순이익', '비지배지분순이익', 'VII. 법인세비용차감전순이익(손실)', '지배기업 소유주',
       'VIII. 당기순이익', 'I. 당기순이익', 'I.당기순이익', 'VII. 법인세비용차감전순이익',
       'VII. 법인세비용차감전순손익', '분기순이익', '반기순이익', '계속영업분기순이익(손실)',
       '법인세비용차감전계속사업이익(손실)', '계속사업순이익(손실)', '총포괄손익', '지배기업소유주지분순이익(손실)',
       '비지배지분순이익(손실)', '계속영업 당기순이익(손실)', '계속기업당기순이익(손실)',
       '지배기업주주지분 계속영업이익', '지배기업주주지분 중단영업이익', '법인비용세차감전순손실', '계속영

In [84]:
(df[(df['항목명'].str.contains('당기순이익'))|(df['항목코드'].str.contains('ifrs_ProfitLoss'))]['항목명'].unique())

array(['법인세비용차감전순이익(손실)', '계속영업당기순이익(손실)', '중단영업당기순이익(손실)', '계속영업이익(손실)',
       '지배기업의 소유주에게 귀속되는 당기순이익(손실)', '비지배지분에 귀속되는 당기순이익(손실)',
       '법인세비용차감전이익(손실)', '당기순이익(손실)', '법인세비용차감전순이익', '지배주주지분', '비지배지분',
       '중단영업이익(손실)', '연결당기순이익', '당기순이익', '연결당기순이익(순손실)', '지배기업주주지분',
       '당기순이익(순손실)', '지배기업의 소유주지분', '지배기업 귀속 반기순이익', '비지배지분 귀속 반기순이익',
       '지배기업의 소유주', '지배기업 소유주지분', '후속적으로 당기순이익으로 재분류 되지않는 항목',
       '후속적으로 당기순이익으로 재분류 되는 항목', '지배기업소유주지분', '지배기업소유주지분순이익', '비지배지분순이익',
       'VII. 법인세비용차감전순이익(손실)', '지배기업 소유주', 'VIII. 당기순이익', 'I. 당기순이익',
       'I.당기순이익', '법인세비용차감전이익', '반기순이익', '계속영업분기순이익(손실)',
       '법인세비용차감전계속사업이익(손실)', '계속사업순이익(손실)', '계속영업 당기순이익(손실)',
       '계속기업당기순이익(손실)', '법인비용세차감전순손실', '계속영업손실', '당기총포괄손실', '비지배기업 소유주지분',
       '계속영업당기순이익', '중단영업당기순이익', '계속영업당기순이익의 귀속', '당기순이익의 귀속',
       '연결당기순이익(손실)', '지배주주지분 당기순이익', '비지배주주지분 당기순이익', '연결계속사업당기순이익(손실)',
       '법인세비용차감전순손익', '계속영업순이익', '포괄손익', '지배기업소유지분', '계속영업연결당기순이익',
       '법인세비용차감전순손익(손실)', '지배기업주주지분당기순이익(손실)

In [78]:
(df[(df['항목명'].str.contains('당기순이익'))]['항목명'].unique())

array(['계속영업당기순이익(손실)', '중단영업당기순이익(손실)', '지배기업의 소유주에게 귀속되는 당기순이익(손실)',
       '비지배지분에 귀속되는 당기순이익(손실)', '당기순이익(손실)', '연결당기순이익', '당기순이익',
       '연결당기순이익(순손실)', '당기순이익(순손실)', '후속적으로 당기순이익으로 재분류 되지않는 항목',
       '후속적으로 당기순이익으로 재분류 되는 항목', 'VIII. 당기순이익', 'I. 당기순이익', 'I.당기순이익',
       '계속영업 당기순이익(손실)', '계속기업당기순이익(손실)', '계속영업당기순이익', '중단영업당기순이익',
       '계속영업당기순이익의 귀속', '당기순이익의 귀속', '연결당기순이익(손실)', '지배주주지분 당기순이익',
       '비지배주주지분 당기순이익', '연결계속사업당기순이익(손실)', '계속영업연결당기순이익',
       '지배기업주주지분당기순이익(손실)', '비지배지분당기순이익(손실)', 'Ⅴ. 당기순이익', '당기순이익 (손실)',
       '당기순이익[손실]', '계속영업 당기순이익', '중단영업 당기순이익', '당기순이익(손실)의 귀속',
       '계속사업지배지분당기순이익', '중단사업지배지분당기순이익', '연결당기순이익의귀속',
       '지배지분에 귀속되는 당기순이익(손실)', 'VI. 당기순이익(손실)', '후속적으로 당기순이익으로 재분류되는 항목',
       '당기순이익(손실).', '계속영업연결당기순이익(손실)', '연결당기순이익귀속', '계속사업연결당기순이익',
       '계속사업당기순이익(손실)', '지배기업의 소유주에 귀속될 당기순이익', '지배기업의 소유주 귀속 당기순이익',
       '비지배지분 귀속 당기순이익', '지배기업의 소유주에 귀속될 당기순이익(손실)', 'Ⅷ.당기순이익(손실)',
       '당기순이익(손실)의 귀속:', '당기순이익과 연결포괄이익', '지배기업의 소유주에게 귀속되는 

In [75]:
df[(df['항목명'].str.contains('비지배지분'))]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
4962,001140,2016,1분기보고서,국보,493,ifrs_ProfitLossAttributableToNoncontrollingInt...,비지배지분에 귀속되는 당기순이익(손실),"-11,627,034"
4967,001140,2016,1분기보고서,국보,493,ifrs_ComprehensiveIncomeAttributableToNoncontr...,"포괄손익, 비지배지분","-11,627,034"
15096,001360,2017,반기보고서,삼성제약,212,entity00126414_udf_IS_2017810124829149_ProfitLoss,비지배지분,"1,088"
15103,001360,2017,반기보고서,삼성제약,212,ifrs_ComprehensiveIncomeAttributableToNoncontr...,비지배지분,"1,064"
17329,001430,2015,사업보고서,세아베스틸,241,ifrs_ProfitLossAttributableToNoncontrollingInt...,비지배지분에 귀속되는 당기순이익(손실),"14,138,747,713"
...,...,...,...,...,...,...,...,...
870,900290,2020,반기보고서,GRT,649,ifrs-full_ProfitLossAttributableToNoncontrolli...,비지배지분,"-100,050"
997,900290,2020,사업보고서,GRT,649,ifrs-full_ProfitLossAttributableToNoncontrolli...,비지배지분,"-489,788"
961,900290,2021,1분기보고서,GRT,649,ifrs-full_ProfitLossAttributableToNoncontrolli...,비지배지분,"-1,864,164"
1063,900290,2021,3분기보고서,GRT,649,ifrs-full_ProfitLossAttributableToNoncontrolli...,비지배지분,"-898,915"


In [ ]:
df[df['항목코드'] == 'ifrs_ProfitLoss']

In [85]:
df2 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(매출액누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df2 = df2.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df2 = df2.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df2

910it [03:49,  3.97it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
37637,000240,2016,1분기보고서,한국타이어월드와이드,715,entity00160047_udf_IS_2016515164639527_Stateme...,영업수익,"63,692,032,878"
37638,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_TotalSellingGeneralAdministrativeExpenses,영업비용,"9,462,501,282"
37639,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_OperatingIncomeLoss,영업이익(손실),"54,229,531,596"
37640,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceIncome,금융수익,"2,592,826,093"
37641,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceCosts,금융원가,"84,855,022"
...,...,...,...,...,...,...,...,...
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7497,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_EarningsPerShareAbstract,주당이익 [abstract],NaN
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [86]:
df2 = df2.replace(0, np.NaN)
df2 = df2.replace('0', np.NaN)
df2.dropna(axis=0, inplace=True)
df2

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
37637,000240,2016,1분기보고서,한국타이어월드와이드,715,entity00160047_udf_IS_2016515164639527_Stateme...,영업수익,"63,692,032,878"
37638,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_TotalSellingGeneralAdministrativeExpenses,영업비용,"9,462,501,282"
37639,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_OperatingIncomeLoss,영업이익(손실),"54,229,531,596"
37640,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceIncome,금융수익,"2,592,826,093"
37641,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceCosts,금융원가,"84,855,022"
...,...,...,...,...,...,...,...,...
7493,950220,2021,반기보고서,네오이뮨텍,701,dart_OtherComprehensiveIncomeNetOfTaxGainsLoss...,순확정부채의 재측정손익(세후기타포괄손익),"3,584"
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [91]:
df2[df2['항목명'].str.contains('매출액')]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
29682,000700,2019,1분기보고서,유수홀딩스,715,entity00163691_udf_IS_20195149831645_Statement...,매출액,"91,329,697,403"
30704,000700,2019,3분기보고서,유수홀딩스,715,entity00163691_udf_IS_20195149831645_Statement...,매출액,"279,266,501,315"
30012,000700,2019,반기보고서,유수홀딩스,715,entity00163691_udf_IS_20195149831645_Statement...,매출액,"191,258,277,055"
33010,000700,2019,사업보고서,유수홀딩스,715,entity00163691_udf_IS_20195149831645_Statement...,매출액,"366,111,471,934"
30763,000700,2020,1분기보고서,유수홀딩스,715,entity00163691_udf_IS_20195149831645_Statement...,매출액,"79,356,749,437"
...,...,...,...,...,...,...,...,...
32795,900140,2016,3분기보고서,코라오홀딩스,715,entity00838500_udf_IS_2015525214515894_Stateme...,매출액,"252,823,370"
32580,900140,2016,반기보고서,코라오홀딩스,715,entity00838500_udf_IS_2015525214515894_Stateme...,매출액,"171,217,063"
34189,900140,2016,사업보고서,코라오홀딩스,715,entity00838500_udf_IS_2015525214515894_Stateme...,매출액,"338,068,515"
33626,900140,2017,1분기보고서,코라오홀딩스,715,entity00838500_udf_IS_2015525214515894_Stateme...,매출액,"77,227,985"


In [97]:
len(df2[df2['항목명'].isin(['매출액'])]['종목코드'].unique())

31

In [94]:
통합재무정보[통합재무정보['종목코드'] == '096760']

,종목코드,사업연도,보고서종류,자산총계,자본총계,자본총계(비지배),부채총계,유동자산,당기순이익,당기순이익(비지배),매출액,현금및현금성자산,감가상각비,영업이익,매출총이익
1247,096760,2021,3분기보고서,"1,063,971,321,688","326,916,645,349","188,725,175,039","737,054,676,340","461,869,988,187","20,276,660,456","11,993,113,669","557,579,206,314","45,254,459,147",0,"41,870,056,048","244,497,809,996"
1248,096760,2021,1분기보고서,"1,084,379,145,682","331,117,672,329","185,246,184,781","753,261,473,353","474,215,304,389","11,330,169,127","5,205,278,874","189,566,826,955","29,319,464,690",0,"20,036,455,536","86,693,592,123"
1249,096760,2021,반기보고서,"1,052,554,347,497","317,209,629,824","181,585,192,565","735,344,717,673","449,493,563,594","10,226,224,651","5,192,836,439","379,700,340,100","25,913,573,338",0,"27,141,956,675","171,578,915,980"
1250,096760,2015,사업보고서,"1,082,862,852,768","270,305,928,996","166,596,435,462","812,556,923,772","572,264,182,636",0,0,0,"15,335,030,317",0,0,"256,511,361,871"
1251,096760,2016,3분기보고서,"1,148,488,597,153","275,510,549,478","189,421,459,230","872,978,047,675","650,145,719,623",0,0,0,"29,634,132,937",0,0,"210,194,661,058"
1252,096760,2016,사업보고서,"1,196,629,708,762","277,590,775,895","161,036,596,392","919,038,932,867","694,619,833,935",0,0,0,"53,216,400,781",0,0,"281,624,597,068"
1253,096760,2016,1분기보고서,"1,115,904,764,705","271,201,800,429","166,386,267,791","844,702,964,276","613,868,893,680",0,0,0,"18,101,773,496",0,0,"70,750,337,750"
1254,096760,2016,반기보고서,"1,120,722,901,718","266,045,804,778","161,318,519,125","854,677,096,940","626,974,739,090",0,0,0,"24,944,648,587",0,0,"140,096,875,484"
1255,096760,2017,3분기보고서,"1,364,242,223,705","378,527,119,935","205,289,781,558","985,715,103,770","671,340,339,513","69,317,641,174","6,537,589,768","537,261,675,995","49,148,950,242",0,"37,151,005,034","229,460,824,813"
1256,096760,2017,사업보고서,"1,351,379,775,398","374,730,109,698","207,147,525,772","976,649,665,700","659,153,431,005","56,869,735,184","2,847,919,403","683,911,348,773","41,620,179,510",0,"41,424,989,829","297,498,379,665"


# 누락데이터 확인

In [10]:
누락데이터 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

sample = 손익계산샘플[손익계산샘플['종목코드'].isin(매출액누락 + 영업이익누락 + 매출총이익누락 + 당기순이익누락)].drop_duplicates()
sample

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
264,032640,2016,사업보고서,LG유플러스,612,ifrs_Revenue,영업수익,"11,451,046,000,000"
265,032640,2016,사업보고서,LG유플러스,612,ifrs_CostOfSales,상품구입원가,"2,597,480,000,000"
266,032640,2016,사업보고서,LG유플러스,612,ifrs_GrossProfit,매출총이익,"8,853,566,000,000"
267,032640,2016,사업보고서,LG유플러스,612,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"8,107,083,000,000"
268,032640,2016,사업보고서,LG유플러스,612,dart_SalariesWages,종업원급여,"806,900,000,000"
...,...,...,...,...,...,...,...,...
4033,008770,2016,3분기보고서,호텔신라,478,ifrs_ProfitLossAttributableToOwnersOfParent,지배기업 소유주,"26,243,250,350"
4034,008770,2016,3분기보고서,호텔신라,478,ifrs_ProfitLossAttributableToNoncontrollingInt...,비지배지분,NaN
4035,008770,2016,3분기보고서,호텔신라,478,ifrs_EarningsPerShareAbstract,IX.주당이익,NaN
4036,008770,2016,3분기보고서,호텔신라,478,ifrs_BasicEarningsLossPerShare,기본및희석보통주당이익,686


In [12]:
누락데이터 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

sample2 = 포괄손익계산샘플[포괄손익계산샘플['종목코드'].isin(매출액누락 + 영업이익누락 + 매출총이익누락 + 당기순이익누락)].drop_duplicates()
sample2

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
24,095570,2016,반기보고서,AJ네트웍스,693,ifrs_Revenue,I. 영업수익,"599,785,915,747"
25,095570,2016,반기보고서,AJ네트웍스,693,ifrs_CostOfSales,II. 영업비용,"567,102,103,937"
26,095570,2016,반기보고서,AJ네트웍스,693,dart_OperatingIncomeLoss,III. 영업이익,"32,683,811,810"
27,095570,2016,반기보고서,AJ네트웍스,693,dart_OtherGains,기타수익,"8,162,334,503"
28,095570,2016,반기보고서,AJ네트웍스,693,dart_OtherLosses,기타비용,"2,854,970,253"
...,...,...,...,...,...,...,...,...
41886,145020,2016,3분기보고서,휴젤,211,ifrs_ComprehensiveIncomeAttributableToOwnersOf...,지배기업의 소유주 귀속 총포괄이익,"29,802,465,246"
41887,145020,2016,3분기보고서,휴젤,211,ifrs_ComprehensiveIncomeAttributableToNoncontr...,비지배지분 귀속 총포괄이익,"5,374,640,361"
41888,145020,2016,3분기보고서,휴젤,211,ifrs_EarningsPerShareAbstract,주당순이익,NaN
41889,145020,2016,3분기보고서,휴젤,211,ifrs_BasicEarningsLossPerShare,기본주당순손익,"9,046"


In [ ]:

# (수익)매출액 (ifrs-full_Revenue)
매출액 = sample[(sample['항목코드'] == 'ifrs_Revenue') | (sample['항목코드'] == 'ifrs-full_Revenue') | (sample['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
매출액.rename(columns={당기:"당기"}, inplace=True)
매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(매출액, sort=False, ignore_index=True)

 # 매출총이익 (ifrs-full_GrossProfit))
매출총이익 = sample[(sample['항목코드'] == 'ifrs_GrossProfit') | (sample['항목코드'] == 'ifrs-full_GrossProfit')| (sample['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
매출총이익.rename(columns={당기:"당기"}, inplace=True)
매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(매출총이익, sort=False, ignore_index=True)

# 영업이익(손실) (dart_OperatingIncomeLoss)
영업이익 = sample[(sample['항목코드'] == 'dart_OperatingIncomeLoss')|(sample['항목명'].isin(['영업이익','영업이익(손실)']))]
영업이익.rename(columns={당기:"당기"}, inplace=True)
영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
누락데이터 = 누락데이터.append(영업이익, sort=False, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
누락데이터

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2016,사업보고서,CJ,715,ifrs_Revenue,영업수익,"23,954,197,397,000"
1,000120,2016,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"6,081,945,642,204"
2,155660,2016,사업보고서,DSR,139,ifrs_Revenue,매출액,"201,060,062,780"
3,078930,2016,사업보고서,GS,715,ifrs_Revenue,매출,"13,462,426,000,000"
4,006360,2016,사업보고서,GS건설,411,ifrs_Revenue,매출액,"11,035,596,402,664"
...,...,...,...,...,...,...,...,...
11399,005380,2016,3분기보고서,현대자동차,301,dart_OperatingIncomeLoss,영업이익,"4,172,334,000,000"
11400,011760,2016,3분기보고서,현대종합상사,468,dart_OperatingIncomeLoss,영업이익,"23,404,203,949"
11401,008770,2016,3분기보고서,호텔신라,478,dart_OperatingIncomeLoss,V.영업이익,"63,348,832,292"
11402,064240,2016,3분기보고서,홈캐스트,265,dart_OperatingIncomeLoss,영업이익(손실),"17,324,059,573"


---

# 데이터 전처리

In [ ]:
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
누락데이터['항목명'] = 누락데이터.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
누락데이터['항목코드'] = 누락데이터.apply(lambda x: 'ifrs-full_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

누락데이터 =  누락데이터.drop_duplicates()

In [ ]:
누락데이터['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss'],
      dtype=object)

In [ ]:
누락데이터['항목명'].unique()

array(['매출액', '매출총이익', '영업이익'], dtype=object)

In [ ]:
누락데이터

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2016,사업보고서,CJ,715,ifrs_Revenue,매출액,"23,954,197,397,000"
1,000120,2016,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"6,081,945,642,204"
2,155660,2016,사업보고서,DSR,139,ifrs_Revenue,매출액,"201,060,062,780"
3,078930,2016,사업보고서,GS,715,ifrs_Revenue,매출액,"13,462,426,000,000"
4,006360,2016,사업보고서,GS건설,411,ifrs_Revenue,매출액,"11,035,596,402,664"
...,...,...,...,...,...,...,...,...
11399,005380,2016,3분기보고서,현대자동차,301,dart_OperatingIncomeLoss,영업이익,"4,172,334,000,000"
11400,011760,2016,3분기보고서,현대종합상사,468,dart_OperatingIncomeLoss,영업이익,"23,404,203,949"
11401,008770,2016,3분기보고서,호텔신라,478,dart_OperatingIncomeLoss,영업이익,"63,348,832,292"
11402,064240,2016,3분기보고서,홈캐스트,265,dart_OperatingIncomeLoss,영업이익,"17,324,059,573"


# 출력

In [ ]:
누락데이터.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/누락데이터.csv')



```javascript
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

```

